In [1]:
#API meteofrance token : eyJ4NXQiOiJZV0kxTTJZNE1qWTNOemsyTkRZeU5XTTRPV014TXpjek1UVmhNbU14T1RSa09ETXlOVEE0Tnc9PSIsImtpZCI6ImdhdGV3YXlfY2VydGlmaWNhdGVfYWxpYXMiLCJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiJqb25haC5qYXViZXJ0QGNhcmJvbi5zdXBlciIsImFwcGxpY2F0aW9uIjp7Im93bmVyIjoiam9uYWguamF1YmVydCIsInRpZXJRdW90YVR5cGUiOm51bGwsInRpZXIiOiJVbmxpbWl0ZWQiLCJuYW1lIjoiQ2FsYW1pdHlDYXN0IiwiaWQiOjE5ODgsInV1aWQiOiJhM2UyNzFlNi1kY2Y4LTQ5N2EtYmE2YS1jODQwNTc2ODhjMWMifSwiaXNzIjoiaHR0cHM6XC9cL3BvcnRhaWwtYXBpLm1ldGVvZnJhbmNlLmZyOjQ0M1wvb2F1dGgyXC90b2tlbiIsInRpZXJJbmZvIjp7fSwia2V5dHlwZSI6IlBST0RVQ1RJT04iLCJwZXJtaXR0ZWRSZWZlcmVyIjoiIiwic3Vic2NyaWJlZEFQSXMiOltdLCJleHAiOjE3Nzk1NDI0ODEsInBlcm1pdHRlZElQIjoiIiwiaWF0IjoxNjg0OTM0NDgxLCJqdGkiOiI1MjI0MTI4MS1mZjAzLTQ0MmUtYjBmMy1iOTU5ZTk2MDUwZTgifQ==.M94PG4zbFjETE8JENhilm8ISop3sl0EpNBTiAevde7ujRO9zc9QPdugbF6Oj_fOrDWW6_yuiAe3kfkL8b9JjdfyvY73y31PUfse6KCRH32iotNDMZJKZcDF7nffBnyraDRDvNyI0GuZiE31I4JCJXjkChDyC54s4-6Ku8ePCfpwrBszTj7_Jy2b45XUgY5L1UKNyfbOeeSVp4r0CfJdn0tkCQLX0guR0CoGmDV_YmglErfvcZ2mvqbypi6fPrX0qvfgHsoTJ4s-5QpKA5Al5JSwY7rNZY_OHu6thCZVrXhbU__nK5q0moNnQv5ep7QBCZuJqMbKE5ciyDhS2lhmWYQ==

In [2]:
pip install meteofrance-api


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
# coding: utf-8
"""Tests Météo-France module."""
import pytest

from meteofrance_api import MeteoFranceClient
from meteofrance_api.helpers import readeable_phenomenoms_dict


@pytest.mark.parametrize("city", ["montreal", "Foix"])
def test_workflow(city: str) -> None:
    """Test classical workflow usage with the Python library."""
    # Init client
    client = MeteoFranceClient()

    # Search a location from name.
    list_places = client.search_places(city)
    my_place = list_places[0]

    # Fetch weather forecast for the location
    my_place_weather_forecast = client.get_forecast_for_place(my_place)

    # Get the daily forecast
    my_place_daily_forecast = my_place_weather_forecast.daily_forecast

    # If rain in the hour forecast is available, get it.
    if my_place_weather_forecast.position["rain_product_available"] == 1:
        my_place_rain_forecast = client.get_rain(my_place.latitude, my_place.longitude)
        next_rain_dt = my_place_rain_forecast.next_rain_date_locale()
        if not next_rain_dt:
            rain_status = "No rain expected in the following hour."
        else:
            rain_status = next_rain_dt.strftime("%H:%M")
    else:
        rain_status = "No rain forecast available."

    # Fetch weather alerts.
    if my_place.admin2:
        my_place_weather_alerts = client.get_warning_current_phenomenoms(
            my_place.admin2
        )
        readable_warnings = readeable_phenomenoms_dict(
            my_place_weather_alerts.phenomenons_max_colors
        )

    assert type(my_place_daily_forecast) == list
    assert rain_status
    assert type(readable_warnings) == dict

In [6]:
test_workflow("montreal")

In [7]:
# coding: utf-8
"""Tests Météo-France module. Forecast class."""
import time
from datetime import datetime

from .const import MOUNTAIN_CITY
from meteofrance_api import MeteoFranceClient
from meteofrance_api.model import Place


def test_forecast_france() -> None:
    """Test weather forecast results from API."""
    client = MeteoFranceClient()

    weather_forecast = client.get_forecast(latitude=48.8075, longitude=2.24028)
    now_ts = int(time.time())

    assert type(weather_forecast.position) == dict
    assert type(weather_forecast.updated_on) == int
    assert "T" in weather_forecast.daily_forecast[0].keys()
    assert "humidity" in weather_forecast.daily_forecast[0].keys()
    assert "rain" in weather_forecast.probability_forecast[0].keys()
    assert "clouds" in weather_forecast.forecast[0].keys()
    assert (
        type(
            weather_forecast.timestamp_to_locale_time(
                weather_forecast.daily_forecast[0]["dt"]
            )
        )
        == datetime
    )
    assert abs(weather_forecast.nearest_forecast["dt"] - now_ts) <= 30 * 60
    assert now_ts - 3600 <= weather_forecast.current_forecast["dt"] <= now_ts
    assert (
        weather_forecast.today_forecast["dt"]
        == weather_forecast.daily_forecast[0]["dt"]
    )


def test_forecast_world() -> None:
    """Test weather forecast results from API."""
    client = MeteoFranceClient()

    weather_forecast = client.get_forecast(latitude=45.5016889, longitude=73.567256)
    now_ts = int(time.time())

    assert type(weather_forecast.position) == dict
    assert type(weather_forecast.updated_on) == int
    assert "T" in weather_forecast.daily_forecast[0].keys()
    assert "humidity" in weather_forecast.daily_forecast[0].keys()
    assert not weather_forecast.probability_forecast
    assert "clouds" in weather_forecast.forecast[0].keys()
    assert (
        type(
            weather_forecast.timestamp_to_locale_time(
                weather_forecast.daily_forecast[0]["dt"]
            )
        )
        == datetime
    )
    assert abs(weather_forecast.nearest_forecast["dt"] - now_ts) == min(
        abs(x["dt"] - now_ts) for x in weather_forecast.forecast
    )
    assert (
        weather_forecast.current_forecast["dt"]
        == weather_forecast.nearest_forecast["dt"]
    )
    assert (
        weather_forecast.today_forecast["dt"]
        == weather_forecast.daily_forecast[0]["dt"]
    )


def test_forecast_place() -> None:
    """Test weather forecast results from API."""
    client = MeteoFranceClient()

    weather_forecast = client.get_forecast_for_place(place=Place(MOUNTAIN_CITY))

    assert type(weather_forecast.position) == dict
    assert type(weather_forecast.updated_on) == int
    assert "T" in weather_forecast.daily_forecast[0].keys()
    assert "humidity" in weather_forecast.daily_forecast[0].keys()
    assert "rain" in weather_forecast.probability_forecast[0].keys()
    assert "clouds" in weather_forecast.forecast[0].keys()
    assert (
        type(
            weather_forecast.timestamp_to_locale_time(
                weather_forecast.daily_forecast[0]["dt"]
            )
        )
        == datetime
    )

ImportError: attempted relative import with no known parent package

In [8]:
"""Uses an API to get the meteorological forecast and prints it"""

import json
import math
import requests
from datetime import date as datetime
from datetime import timedelta

def get_meteo() -> dict:
    """
        Args: None

        Return: Dictionary that contains the amout of rain per day (date format : yyyy-mm-dd)

        Requests a public API for the meteorological forecast (over a week including yesterday).
        Prints out the detailled results and then prints out the day per day
        And returns the dictionary containing theeses values
    """

    url = "http://www.infoclimat.fr/public-api/gfs/json\
    ?_ll=48.85341,2.3488&_auth=AhhTRFMtBCYDLgA3D3lReFI6BjMPeQMkVysKaVo%2FUy4DaA\
    BhAWFTNQdpUC0HKAo8V3pXNAE6AzNTOAN7CHpXNgJoUz9TOARjA2wAZQ8gUXpSfAZnDy8DJFc8C\
    m9aKVM2A2UAegFjUzIHbVAsBzYKNldlVygBIQM6UzYDZwhhVzMCYFMzUzgEbwNkAH0PIFFjUmAG\
    YA9mA2hXMQpsWjVTZQNmADcBYFM0B2BQLAc2CjxXYlcxATwDPlM2A2wIelcrAhhTRFMtBCYDLgA\
    3D3lReFI0BjgPZA%3D%3D&_c=bc0b6c355d4a1f89b826625db56f406f"

    res = requests.get(url = url, allow_redirects = True)
    data = json.loads(res.text) #Dictionary containg the result of the API call

    if data["request_state"] == "200" or data["message"] == "OK":
        #for key in list(data.keys())[5:]:
            #print(f"date : {key}  ==> pluie : {data[key]['pluie']}")

        #print("Total de pluie pour la journée (en mm) :")
        date = list(data.keys())[5].split(" ")[0]
        tot = float(data[list(data.keys())[5]]["pluie"])
        rain = {}

        for key in list(data.keys())[6:]:
            if date != key.split(" ")[0]:
                rain[key.split(" ")[0]] = math.ceil(tot)
                #print(f"{date} : {math.ceil(tot)} mm")
                date = key.split(" ")[0]
                tot = 0
            tot += float(data[key]['pluie'])

        rain[key] = math.ceil(tot)
        #print(f"{date} : {math.ceil(tot)} mm")

    return rain

def diff_dates(date1 : str, date2 = str(datetime.today())):
    nb_days = { "01" : 31,
                "02" : 28,
                "03" : 31,
                "04" : 30,
                "05" : 31,
                "06" : 30,
                "07" : 31,
                "08" : 31,
                "09" : 30,
                "10" : 31,
                "11" : 30,
                "12" : 31,
                }

    date1 = date1.split("-")
    date2 = date2.split("-")
    years = int(date2[0]) - int(date1[0])
    months = int(date2[1]) - int(date1[1])
    days = int(date2[2]) - int(date1[2])
    months = months + years * 12
    days = days + months * nb_days[date2[1]]
    return days


def write_to_json(path : str):
    url = "http://www.infoclimat.fr/public-api/gfs/json\
    ?_ll=48.85341,2.3488&_auth=AhhTRFMtBCYDLgA3D3lReFI6BjMPeQMkVysKaVo%2FUy4DaA\
    BhAWFTNQdpUC0HKAo8V3pXNAE6AzNTOAN7CHpXNgJoUz9TOARjA2wAZQ8gUXpSfAZnDy8DJFc8C\
    m9aKVM2A2UAegFjUzIHbVAsBzYKNldlVygBIQM6UzYDZwhhVzMCYFMzUzgEbwNkAH0PIFFjUmAG\
    YA9mA2hXMQpsWjVTZQNmADcBYFM0B2BQLAc2CjxXYlcxATwDPlM2A2wIelcrAhhTRFMtBCYDLgA\
    3D3lReFI0BjgPZA%3D%3D&_c=bc0b6c355d4a1f89b826625db56f406f"

    res = requests.get(url = url, allow_redirects = True)
    data = json.loads(res.text)
    final = {"0": {}, "1":{}, "2":{}}
    final_temp = {"0": {}, "1":{}, "2":{}}

    if data["request_state"] == "200" or data["message"] == "OK":
        keys = list(data.keys())[6:]
        for key in keys:
            date, hour = key.split(" ")
            diff = -diff_dates(date)
            if diff >= 0 and diff <= 2:
                final[str(diff)][hour.split(":")[0]] = data[date + " " + hour]["pluie"]
                final_temp[str(diff)][hour.split(":")[0]] = int(data[date + " " + hour]["temperature"]["sol"] - 273)

    with open(path + "/pluie.json", 'r') as file:
        temp = json.loads(file.read())
        for key in list(temp.keys())[:6]:
            if key not in data.keys():
                final[key] = temp[key]

    with open(path + "/temperature.json", 'r') as file:
        temp = json.loads(file.read())
        for key in list(temp.keys())[:6]:
            if key not in data.keys():
                final_temp[key] = temp[key]

    with open(path + "/pluie.json", 'w') as file:
        json.dump(final, file)

    with open(path + "/temperature.json", 'w') as file:
        json.dump(final_temp, file)
    return final, final_temp

def main():

    """
    Description : main function is called only if the file is called directly
    """

    get_meteo()
    #meteo = get_meteo()
    #print(dict)

if __name__ == '__main__':
    main()

In [9]:
main()

In [10]:
get_meteo()

{'2023-05-25': 1,
 '2023-05-26': 1,
 '2023-05-27': 0,
 '2023-05-28': 0,
 '2023-05-29': 0,
 '2023-05-30': 0,
 '2023-05-31': 1,
 '2023-06-01': 1,
 '2023-06-01 02:00:00': 0}